<a href="https://colab.research.google.com/github/kr7/ROCKET-DTW/blob/main/ROCKET_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Activity Recognition Based on Accelerometer Data with Enhanced ROCKET Algorithm**

In [ ]:
import numpy as np
from numpy import random as rnd
from scipy.io import arff
from scipy.stats import ttest_rel
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeClassifier

In [ ]:
!wget https://timeseriesclassification.com/aeon-toolkit/Epilepsy.zip

--2024-02-07 09:36:52--  https://timeseriesclassification.com/aeon-toolkit/Epilepsy.zip
Resolving timeseriesclassification.com (timeseriesclassification.com)... 109.123.71.232
Connecting to timeseriesclassification.com (timeseriesclassification.com)|109.123.71.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 809010 (790K) [application/zip]
Saving to: ‘Epilepsy.zip’

Epilepsy.zip        100%[===================>] 790.05K  1.61MB/s    in 0.5s    

2024-02-07 09:36:53 (1.61 MB/s) - ‘Epilepsy.zip’ saved [809010/809010]



In [ ]:
!unzip Epilepsy.zip

Archive:  Epilepsy.zip
  inflating: Epilepsy.jpg            
  inflating: Epilepsy.txt            
  inflating: EpilepsyDimension1_TEST.arff  
  inflating: EpilepsyDimension1_TRAIN.arff  
  inflating: EpilepsyDimension2_TEST.arff  
  inflating: EpilepsyDimension2_TRAIN.arff  
  inflating: EpilepsyDimension3_TEST.arff  
  inflating: EpilepsyDimension3_TRAIN.arff  
  inflating: Epilepsy_TEST.arff      
  inflating: Epilepsy_TRAIN.arff     
  inflating: Epilepsy_TEST.ts        
  inflating: Epilepsy_TRAIN.ts       


In [ ]:
# In order to perform 10-fold cross-validation, we
# will merge the provided train and test splits and
# we will split the data durign the cross-validation

NUM_CHANNELS = 3

X = []
y = []

for filename in ['Epilepsy_TRAIN.arff', 'Epilepsy_TEST.arff' ]:
  raw_data = arff.loadarff(filename)

  nextInstance = True
  i = 0

  while nextInstance:
    try:
      dim1 = list(raw_data[0][i][0][0])
      dim2 = list(raw_data[0][i][0][1])
      dim3 = list(raw_data[0][i][0][2])
      label = raw_data[0][i][1]

      X.append([dim1,dim2,dim3])
      if label == b'EPILEPSY':
        label = 0
      elif label == b'WALKING':
        label = 1
      elif label == b'RUNNING':
        label = 2
      elif label == b'SAWING':
        label = 3
      else:
        print(f'Unexpected label: {label}')
      y.append(label)

      i = i+1
    except:
      nextInstance = False

X = np.array(X)
y = np.array(y)

In [ ]:
%load_ext cython

In [ ]:
%%cython

cimport cython
from libc.stdlib cimport malloc, free


def dtw(ts1p, ts2p):
    cdef int LEN_TS1
    cdef int LEN_TS2
    cdef int i
    cdef int j
    cdef float * ts1
    cdef float * ts2
    cdef float * dtw_matrix
    cdef float d

    LEN_TS1 = len(ts1p)
    LEN_TS2 = len(ts2p)

    ts1 = <float *> malloc(LEN_TS1*cython.sizeof(float))
    ts2 = <float *> malloc(LEN_TS2*cython.sizeof(float))
    dtw_matrix = <float *> malloc(LEN_TS1*LEN_TS2*cython.sizeof(float))
      # this is a flattened DTW matrix dtw_matrix[i,j] -> dtw_matrix[i*LEN_TS2 + j]
    if ts1 is NULL or ts2 is NULL:
      raise MemoryError()
    for i in xrange(LEN_TS1):
      ts1[i] = ts1p[i]
    for i in xrange(LEN_TS2):
      ts2[i] = ts2p[i]

    dtw_matrix[0] = abs(ts1[0]-ts2[0])

    for i in range(1, LEN_TS1):
      dtw_matrix[i*LEN_TS2] = dtw_matrix[(i-1)*LEN_TS2]+abs(ts1[i]-ts2[0])

    for j in range(1, LEN_TS2):
      dtw_matrix[j] = dtw_matrix[j-1]+abs(ts1[0]-ts2[j])

    for i in range(1, LEN_TS1):
      for j in range(1, LEN_TS2):
        dtw_matrix[i*LEN_TS2+j] = min(dtw_matrix[(i-1)*LEN_TS2+j-1],
                                      dtw_matrix[(i-1)*LEN_TS2+j],
                                      dtw_matrix[i*LEN_TS2+j-1]) + abs(ts1[i]-ts2[j])

    d=dtw_matrix[ LEN_TS1*LEN_TS2-1 ]

    free(ts1)
    free(ts2)
    free(dtw_matrix)

    return d

In [ ]:
def log2(x):
  return np.log(x)/np.log(2)


def get_random_convolution_params(seed, l_input):
  rnd.seed(seed)
  length = rnd.randint(0,2)*2+7
  weights = rnd.normal(0,1, (NUM_CHANNELS, length) )
  bias = float(rnd.uniform(-1, 1))
  max_exp = np.floor(log2((l_input-1)/(length-1)))-1
  dilation = int( 2**(rnd.uniform(0,max_exp)) )
  padding = rnd.randint(0,1)
  return (length, weights, bias, dilation, padding)


def apply_convolution( time_series, convolution ):
  length, weights, bias, dilation, padding = convolution
  window_size = length*dilation
  time_series = list(time_series)

  if padding == 1:
    zeros = [0]*(dilation*(length-1)/2)
    for i in range(NUM_CHANNELS):
      time_series[i] = zeros + time_series[i] + zeros

  segments = np.array(
      [ [ time_series[j][i:i+window_size:dilation] for j in range(NUM_CHANNELS) ]
        for i in range(0, int(len(time_series[0])-window_size)) ])

  conv = np.array([np.sum(s*weights) + bias for s in segments])
  dtw_conv = np.array( [ np.sum([dtw(s[c], weights[c]) for c in range(len(s))]) for s in segments] )

  return conv, dtw_conv


def ppv(series):
  return float(np.sum(series > 0) / np.size(series))


def get_rocket_features(time_series_dataset, convolutional_filters):
  dataset_features_max = [] # features with conventional convolution
  dataset_features_ppv = [] # features with conventional convolution
  dataset_features_dtw = [] # features with dynamic convolution
  i = 0
  for ts in time_series_dataset:
    if i%10==0:
      print(f"{i:4}/{len(time_series_dataset)}")
    i = i+1

    ts_features_max = []
    ts_features_ppv = []
    ts_features_dtw = []
    for c in convolutional_filters:
      convolved_ts, dtw_convolved_ts = apply_convolution(ts, c)
      ts_features_max.append( float(max(convolved_ts)) )
      ts_features_ppv.append( ppv(convolved_ts) )
      ts_features_dtw.append( float(max(dtw_convolved_ts)) )
       # PPV does not make sense for dTW!
    dataset_features_max.append( ts_features_max )
    dataset_features_ppv.append( ts_features_ppv )
    dataset_features_dtw.append( ts_features_dtw )
  return np.array(dataset_features_max), np.array(dataset_features_ppv), np.array(dataset_features_dtw)

In [ ]:
# Convolutional filters are independent of the data (as their parameters are
# sampled from random distributions), therefore the features based on
# convolutional filters may be pre-calculated (so that they do not need to be
# calculated in each round of the cross-validation which speeds up the
# execution of the experiment)

convolutional_filters = []
length = len(X[0][0])
for i in range(10000):
  convolutional_filters.append(get_random_convolution_params(i, length))

features_max, features_ppv, features_dtw = get_rocket_features(X, convolutional_filters)

   0/275
  10/275
  20/275
  30/275
  40/275
  50/275
  60/275
  70/275
  80/275
  90/275
 100/275
 110/275
 120/275
 130/275
 140/275
 150/275
 160/275
 170/275
 180/275
 190/275
 200/275
 210/275
 220/275
 230/275
 240/275
 250/275
 260/275
 270/275


In [ ]:
# 10-times 10-fold cross-validation

all_err_rocket = np.zeros( (100) )
all_err_ppv = np.zeros( (100) )
all_err_max = np.zeros( (100) )
all_err_dtw = np.zeros( (100) )
all_err_combined = np.zeros( (100) )
fold = 0

for seed in range(10):
  kf = StratifiedKFold(n_splits=10, random_state=seed+42, shuffle=True)
  for train_index, test_index in kf.split(X, y):
    print(f"FOLD: {fold:2}")
    features_train_max = features_max[train_index]
    features_train_ppv = features_ppv[train_index]
    features_train_dtw = features_dtw[train_index]
    features_test_max = features_max[test_index]
    features_test_ppv = features_ppv[test_index]
    features_test_dtw = features_dtw[test_index]

    y_train = y[train_index]
    y_test  = y[test_index]

    cls = RidgeClassifier()
    cls.fit( np.hstack((features_train_max, features_train_ppv)), y_train)
    pred = cls.predict( np.hstack((features_test_max, features_test_ppv)) )
    all_err_rocket[fold] = np.mean(pred != y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_ppv, y_train )
    pred = cls.predict( features_test_ppv )
    all_err_ppv[fold] = np.mean(pred != y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_max, y_train )
    pred = cls.predict( features_test_max )
    all_err_max[fold] = np.mean(pred != y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_dtw, y_train )
    pred = cls.predict( features_test_dtw )
    all_err_dtw[fold] = np.mean(pred != y_test)

    cls = RidgeClassifier()
    cls.fit( np.hstack((features_train_max, features_train_ppv, features_train_dtw)), y_train)
    pred = cls.predict( np.hstack((features_test_max, features_test_ppv, features_test_dtw)) )
    all_err_combined[fold] = np.mean(pred != y_test)

    fold = fold + 1

FOLD:  0
FOLD:  1
FOLD:  2
FOLD:  3
FOLD:  4
FOLD:  5
FOLD:  6
FOLD:  7
FOLD:  8
FOLD:  9
FOLD: 10
FOLD: 11
FOLD: 12
FOLD: 13
FOLD: 14
FOLD: 15
FOLD: 16
FOLD: 17
FOLD: 18
FOLD: 19
FOLD: 20
FOLD: 21
FOLD: 22
FOLD: 23
FOLD: 24
FOLD: 25
FOLD: 26
FOLD: 27
FOLD: 28
FOLD: 29
FOLD: 30
FOLD: 31
FOLD: 32
FOLD: 33
FOLD: 34
FOLD: 35
FOLD: 36
FOLD: 37
FOLD: 38
FOLD: 39
FOLD: 40
FOLD: 41
FOLD: 42
FOLD: 43
FOLD: 44
FOLD: 45
FOLD: 46
FOLD: 47
FOLD: 48
FOLD: 49
FOLD: 50
FOLD: 51
FOLD: 52
FOLD: 53
FOLD: 54
FOLD: 55
FOLD: 56
FOLD: 57
FOLD: 58
FOLD: 59
FOLD: 60
FOLD: 61
FOLD: 62
FOLD: 63
FOLD: 64
FOLD: 65
FOLD: 66
FOLD: 67
FOLD: 68
FOLD: 69
FOLD: 70
FOLD: 71
FOLD: 72
FOLD: 73
FOLD: 74
FOLD: 75
FOLD: 76
FOLD: 77
FOLD: 78
FOLD: 79
FOLD: 80
FOLD: 81
FOLD: 82
FOLD: 83
FOLD: 84
FOLD: 85
FOLD: 86
FOLD: 87
FOLD: 88
FOLD: 89
FOLD: 90
FOLD: 91
FOLD: 92
FOLD: 93
FOLD: 94
FOLD: 95
FOLD: 96
FOLD: 97
FOLD: 98
FOLD: 99


In [ ]:
print( "Method                    Classification error  p-values")
print(f"ROCKET                    "+
      f"{np.mean(all_err_rocket):6.4f} +/- {np.std(all_err_rocket):6.4f}  ")
print(f"ROCKET-PPV                "+
      f"{np.mean(all_err_ppv):6.4f} +/- {np.std(all_err_ppv):6.4f}  ")
print(f"ROCKET-MAX                "+
      f"{np.mean(all_err_max):6.4f} +/- {np.std(all_err_max):6.4f}  ")
print(f"ROCKET-DTW (our approach) "+
      f"{np.mean(all_err_dtw):6.4f} +/- {np.std(all_err_dtw):6.4f}     "+
      f"{ttest_rel(all_err_rocket, all_err_dtw)[1]:5.3f}  "+
      f"{ttest_rel(all_err_ppv, all_err_dtw)[1]:5.3f}  "+
      f"{ttest_rel(all_err_max, all_err_dtw)[1]:5.3f}  ")
print(f"ROCKET-Combined           "+
      f"{np.mean(all_err_combined):6.4f} +/- {np.std(all_err_combined):6.4f}     "+
      f"{ttest_rel(all_err_rocket, all_err_combined)[1]:5.3f}  "+
      f"{ttest_rel(all_err_ppv, all_err_combined)[1]:5.3f}  "+
      f"{ttest_rel(all_err_max, all_err_combined)[1]:5.3f}  "+
      f"{ttest_rel(all_err_dtw, all_err_combined)[1]:5.3f}  ")

Method                    Classification error  p-values
ROCKET                    0.0174 +/- 0.0220  
ROCKET-PPV                0.0154 +/- 0.0254  
ROCKET-MAX                0.0261 +/- 0.0318  
ROCKET-DTW (our approach) 0.0018 +/- 0.0079     0.000  0.000  0.000  
ROCKET-Combined           0.0047 +/- 0.0121     0.000  0.000  0.000  0.004  
